In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import csv
import pandas as pd
from scipy import stats
import re
from scipy.stats import pearsonr
#import rpy2.robjects as robjects
import random
from statsmodels.stats.multitest import fdrcorrection
import copy
from collections import Counter
import seaborn as sns
from scipy.stats import binom_test
import gseapy as gs
from scipy.stats import spearmanr
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_rel

#Read in sfari genes
sfari = pd.read_csv("SFARI-Gene_genes_03-28-2024release_05-09-2024export.csv")
#sfari = sfari[sfari["gene-score"] == 1]
sfari = {"SFARI":list(sfari["gene-symbol"])}

In [ ]:
#Indistinguishable Ka/Ks for this study
adj = 0.01
v = pd.read_csv("KA_KS_Stuff.csv")
v = v[["Symbol", "Ka (Human)", "Ks (Human)", "Ka (Chimpanzee)", "Ks (Chimpanzee)"]]
v["Ka/Ks human"] = (v["Ka (Human)"]+adj)/(v["Ks (Human)"]+adj)
v["Ka/Ks chimp"] = (v["Ka (Chimpanzee)"]+adj)/(v["Ks (Chimpanzee)"]+adj)
v = v[(v["Ka/Ks human"] >= 0) & (v["Ka/Ks human"] < 100000)]
v = v[(v["Ka/Ks chimp"] >= 0) & (v["Ka/Ks chimp"] < 100000)]
vsf = v[v["Symbol"].isin(sfari["SFARI"])]
print(mwu(v["Ka/Ks human"], v["Ka/Ks chimp"]))
print(mwu(vsf["Ka/Ks human"], vsf["Ka/Ks chimp"]))
print(np.median(v["Ka/Ks human"]), np.median(v["Ka/Ks chimp"]))
print(np.median(vsf["Ka/Ks human"]), np.median(vsf["Ka/Ks chimp"]))

In [ ]:
#Repeating analysis or dN/dS in this study, only this one is shown since it is probably better than the previous one

v = pd.read_csv("dN_dS_2014_BMC_Genomics.csv")
ids = pd.read_csv("HumanID_Mapping.txt", sep = "\t")
ids = ids.set_index("Protein stable ID")
v = v.set_index("Human Protein")
v = v[['dN Human', 'dS Human', 'dN Chimpanzee', 'dS Chimpanzee', 'dN/dS Human', 'dN/dS Chimpanzee']]
v = v.join(ids).dropna().drop_duplicates("Gene name")
v = v[(v["dN Human"] > 0) & (v["dS Human"] > 0) & (v["dN Chimpanzee"] > 0) & (v["dN Chimpanzee"] > 0)]
#dN/dS from https://bmcgenomics.biomedcentral.com/articles/10.1186/1471-2164-15-599#Sec18
print(np.median(v['dN/dS Human']), np.median(v['dN/dS Chimpanzee']))
print(mwu(v['dN/dS Human'], v['dN/dS Chimpanzee']))
v_sfari = v[v["Gene name"].isin(sfari["SFARI"])]
print(np.median(v_sfari['dN/dS Human']), np.median(v_sfari['dN/dS Chimpanzee']))
print(ttest_rel(v_sfari['dN/dS Human'], v_sfari['dN/dS Chimpanzee']))

In [ ]:
#Plotting dN/dS

sns.set(font_scale = 1.6)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (7, 5))
sns.set_style("white")
new_palette = {"dN/dS Human":"#FF2C0C", "dN/dS Chimp":"#0058FF"}
v["dN/dS Chimp"] = v["dN/dS Chimpanzee"]
v = v[(v["dN/dS Human"] < 2) & (v["dN/dS Chimpanzee"] < 2)]
v_other = v[v["ASD status"].isin(["Not linked to ASD"])]
v_asd = v[v["ASD status"].isin(["ASD-linked"])]
sns.kdeplot(v_asd[["dN/dS Human", "dN/dS Chimp"]], fill = True, palette = new_palette)
plt.xlabel("dN/dS")
#sns.kdeplot(v_other[["dN/dS Human", "dN/dS Chimpanzee"]], fill = True)

In [ ]:
#Can remove 5000 to make it just within 2000 bases of the TSS
from scipy.stats import binomtest
v = pd.read_csv("Human_Promoters_Ortho_Sorted_hg38_NearProm_5000.bed", sep = "\t", header = None)
v["Pos"] = v[0] + v[2].astype(str)
v = v.drop_duplicates("Pos")
vc = pd.read_csv("Human_Promoters_Ortho_Sorted_hg38_NearProm_ChpDer_5000.bed", sep = "\t", header = None)
vc["Pos"] = vc[0] + vc[2].astype(str)
vc = vc.drop_duplicates("Pos")
v_asd = v[v[4].isin(sfari["SFARI"])]
vc_asd = vc[vc[4].isin(sfari["SFARI"])]
num_asd = v_asd.shape[0]
num_asdc = vc_asd.shape[0]
num = v.shape[0]
numc= vc.shape[0]
print(binomtest(num_asd, num_asd + num_asdc, p=num/(num + numc)))
print((num_asd/(num_asd + num_asdc))/(num/(num + numc)))

In [ ]:
#Plot the number of mutations with 5000 bases of the TSS

from scipy.stats import ttest_rel
v_asd_down = v_asd.sample(frac = numc/num, replace = False)
x = Counter(v_asd_down[6])
out = []
for key in x.keys():
    out.append([key, np.log10(x[key])])
df = pd.DataFrame(out).set_index(0)
df.columns = ["Human-derived"]
x = Counter(vc_asd[6])
out = []
for key in x.keys():
    out.append([key, np.log10(x[key])])
df2 = pd.DataFrame(out).set_index(0)
df2.columns = ["Chimp-derived"]
df = df.join(df2).fillna(0)
ttest_rel(df["Human-derived"], df["Chimp-derived"])

sns.set(font_scale = 1.2)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (7, 5))
new_palette = {"Human-derived":"#FF2C0C", "Chimp-derived":"#0058FF"}

ax = sns.kdeplot(df, fill = True, palette = new_palette)
#ax.legend(frameon=False)
plt.xticks([0, 1, 2], [1, 10, 100])
plt.xlabel("Number of mutations within 5 kb of TSS")
plt.show()


In [ ]:
#Computing variance of ASD-linked genes in humans and chimps relative to similarly expressed genes

def var_nz(x):
    x = x[x!=0]
    return np.var(x)

sfari = pd.read_csv("SFARI-Gene_genes_03-28-2024release_05-09-2024export.csv")
#sfari = sfari[sfari["gene-score"] == 1]
sfari = {"SFARI":list(sfari["gene-symbol"])}

#Need to do it purely within sample
CPM = True
folder = "../../Allen_MTG_Single_Cell/Subclass/Processed"
#folder = "../../Ma_Sestan_Single_Cell/Processed"
if CPM:
    skel = "DESeq2_REPLACE_Human_Chimp_cpm.txt"
else:
    skel = "DESeq2_REPLACE_Human_Chimp_tpm.txt"

t = "L2-3_IT"
file = folder + "/" + skel.replace("REPLACE", t)
v = pd.read_csv(file, sep = "\t")
hum_cols = []
chp_cols = []
species = "Human"
for i in v.columns:
    if "Chimp" in i or "PTB" in i:
        chp_cols.append(i)
    elif "Human" in i or "HSB" in i:
        hum_cols.append(i)
if species == "Chimp":
    v["Mean " + species] = np.mean(v[chp_cols], axis = 1)
    v = v.drop(chp_cols, axis = 1)
    
elif species == "Human":
    v["Mean " + species] = np.mean(v[hum_cols], axis = 1)
    v = v.drop(hum_cols, axis = 1)

add_spec = ["Rhesus", "Marmoset"]
if "Allen" in folder:
    add_spec = ["Gorilla"] + add_spec
for j in add_spec:
    vv = pd.read_csv(file.replace("Chimp", j), sep = "\t")
    vv = vv.drop(hum_cols, axis = 1)
    v = v.join(vv)
    gor_cols = []
    if j == "Gorilla" and species == "Gorilla":
        for i in v.columns:
            if "Gorilla" in i:
                gor_cols.append(i)
        v["Mean " + species] = np.mean(v[gor_cols], axis = 1)
        v = v.drop(gor_cols, axis = 1)
v = v.dropna()


means = v[["Mean " + species]].copy()
if species == "Chimp":
    spec = ["Human", "Gorilla", "Rhesus", "Marmoset"]
elif species == "Human":
    spec = ["Chimp", "Gorilla", "Rhesus", "Marmoset"]
elif species == "Gorilla":
    spec = ["Chimp", "Human", "Rhesus", "Marmoset"]
print(spec)
for j in spec:
    j_cols = []
    for i in v.columns:
        if j in i:
            j_cols.append(i)
    means["Mean " + j] = np.mean(v[j_cols], axis = 1)

In [ ]:
#Making sure things are actually correlated

if CPM:
    skel = "DESeq2_REPLACE_Human_Chimp_cpm.txt"
else:
    skel = "DESeq2_REPLACE_Human_Chimp_tpm.txt"
    
if "Allen" in folder:
    celltypes = ['L2/3 IT', 'Sst', 'Sst Chodl', 'OPC', 'Micro-PVM', 'L6 CT', 'Lamp5_Lhx6', 'Chandelier', 'Pvalb', 'L4 IT', 'VLMC', 'Pax6', 'Endo', 'Vip', 'Astro', 'L5 IT', 'L5/6 NP', 'Lamp5', 'Sncg', 'L5 ET', 'Oligo', 'L6 IT Car3', 'L6 IT', 'L6b']
    celltypes = [x.replace(" ", "_").replace("/", "-") for x in celltypes]
else:
    celltypes = ['L2-3_IT', 'PVALB', 'SST_NPY', 'Astro', 'L5-6_NP', 'RB', 'VIP', 'Endo', 'SST_HGF', 'Immune', 'L6_CT', 'L6_IT-1', 'L3-5_IT-3', 'Micro', 'L6_IT-2', 'Oligo', 'L5_ET', 'L3-5_IT-1', 'ADARB2_KCNG1', 'PVALB_ChC', 'OPC', 'L3-5_IT-2', 'VLMC', 'SMC', 'L6B', 'SST', 'LAMP5_RELN', 'PC', 'LAMP5_LHX6']
t = "L2-3_IT"
hum_cols = []
chp_cols = []
file = folder + "/" + skel.replace("REPLACE", t)
v = pd.read_csv(file, sep = "\t")
for i in list(v.columns):
    if "Human" in i or "HSB" in i:
        hum_cols.append(i)
    else:
        chp_cols.append(i)
v["Variance human"] = np.var(v[hum_cols], axis = 1)
v["Variance chimp"] = np.var(v[chp_cols], axis = 1)
v["Mean human"] = np.mean(v[hum_cols], axis = 1)
v["Mean chimp"] = np.mean(v[chp_cols], axis = 1)
v = v[(v["Mean human"] > 1) & (v["Mean chimp"] > 1)]
v["Adj var human"] = v["Variance human"]/v["Mean human"]
v["Adj var chimp"] = v["Variance chimp"]/v["Mean chimp"]
v = v.dropna()
print(spearmanr(v["Adj var human"], v["Adj var chimp"]))
print(np.median(v["Adj var human"]), np.median(v["Adj var chimp"]))
v_asd = v.loc[np.intersect1d(sfari["SFARI"], v.index)]
print(spearmanr(v_asd["Adj var human"], v_asd["Adj var chimp"]))
print(np.median(v_asd["Adj var human"]), np.median(v_asd["Adj var chimp"]))
sns.regplot(x=v_asd["Adj var human"], y=v_asd["Adj var chimp"])

In [ ]:
hum_df = v[hum_cols]
chp_df = v[chp_cols]

hum_df_var = pd.DataFrame(hum_df.apply(np.var, axis = 1))
hum_df_var.columns = ["Variance"]
hum_df_var["Variance no zeros"] = hum_df.apply(var_nz, axis = 1)

chp_df_var = pd.DataFrame(chp_df.apply(np.var, axis = 1))
chp_df_var.columns = ["Variance"]
chp_df_var["Variance no zeros"] = chp_df.apply(var_nz, axis = 1)

In [ ]:
means = means.loc[np.intersect1d(hum_df.index, means.index)]
means = means.sort_values("Mean Human")
means["Rank"] = list(range(means.shape[0]))
analyze = np.intersect1d(means.index, sfari["SFARI"])
d = {}
for gene in analyze:
    rank = means.loc[gene]["Rank"]
    if rank < means.shape[0] - 50 and rank > 50:
        other = list(range(int(rank) - 50, int(rank) + 50))
        other.remove(rank)
        other_genes = list(means[means["Rank"].isin(other)].index)
        d[gene] = other_genes

In [ ]:
out_hum = []
out_chp = []
for gene in d.keys():
    hum_df_var_gene = hum_df_var.loc[gene]
    hum_df_var_other = hum_df_var.loc[d[gene]].copy()
    num_less_variance = hum_df_var_other[hum_df_var_other["Variance"] < hum_df_var_gene["Variance"]].shape[0]
    num_less_variance_nz = hum_df_var_other[hum_df_var_other["Variance no zeros"] < hum_df_var_gene["Variance no zeros"]].shape[0]
    out_hum.append([gene, num_less_variance, num_less_variance_nz])
    
    chp_df_var_gene = chp_df_var.loc[gene]
    chp_df_var_other = chp_df_var.loc[d[gene]].copy()
    num_less_variance = chp_df_var_other[chp_df_var_other["Variance"] < chp_df_var_gene["Variance"]].shape[0]
    num_less_variance_nz = chp_df_var_other[chp_df_var_other["Variance no zeros"] < chp_df_var_gene["Variance no zeros"]].shape[0]
    out_chp.append([gene, num_less_variance, num_less_variance_nz])

df_var_hum = pd.DataFrame(out_hum)
df_var_hum.columns = ["Gene", "Num less var hum", "Num less var nz hum"]
df_var_hum = df_var_hum.set_index('Gene')
df_var_chp = pd.DataFrame(out_chp)
df_var_chp.columns = ["Gene", "Num less var chp", "Num less var nz chp"]
df_var_chp = df_var_chp.set_index('Gene')
df_var = df_var_hum.join(df_var_chp)

sns.regplot(x=df_var["Num less var hum"], y = df_var["Num less var chp"])
print(spearmanr(df_var["Num less var hum"], df_var["Num less var chp"]))

In [ ]:
#Actually do binomial test

hum_more_var = df_var[df_var["Num less var hum"] > df_var["Num less var chp"]].shape[0]
chp_more_var = df_var[df_var["Num less var hum"] < df_var["Num less var chp"]].shape[0]
print(binom_test(hum_more_var, hum_more_var + chp_more_var), hum_more_var, chp_more_var)

In [ ]:
#Make plots, can change to Medial temporal gyrus as needed
ttest_rel(df_var["Num less var hum"], df_var["Num less var chp"])
df_var["Human"] = df_var["Num less var hum"]
df_var["Chimp"] = df_var["Num less var chp"]

sns.set(font_scale = 1.6)
sns.set_style("white")
fig, ax = plt.subplots(figsize = (7, 5))
sns.set_style("white")

new_palette = {"Human":"#FF2C0C", "Chimp":"#0058FF"}

sns.histplot(df_var[["Human", "Chimp"]], fill = True, palette = new_palette)
plt.xlabel("Percentage of similarly expressed genes\nwith less variance than ASD-linked gene")
ax.legend(frameon=False)
plt.title("Dorsolateral prefrontal cortex")
#sns.kdeplot(v_other[["dN/dS Human", "dN/dS Chimpanzee"]], fill = True)